# **Langchain: Reterival Augmented Generation**

In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


## **Loading the Document Using PyPDFLoader**

In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 1.1 MB/s eta 0:00:00


In [4]:
from langchain.document_loaders import PyPDFLoader
file_path = '/content/National AI Policy Consultation Draft V1.pdf'
loader = PyPDFLoader(file_path)
pages = loader.load()

In [5]:
print(len(pages))

41


## **Cleaning the Pages data removing unnessary spaces**

In [6]:
cleaned_data = []
for page in pages:
  text = page.page_content
  cleaned_text = " ".join(text.split())
  page.page_content = cleaned_text
  cleaned_data.append(page)

In [7]:
print(len(cleaned_data))

41


# **MetaData Preprocessing and Splitting the data**

In [10]:
from langchain.docstore.document import Document

In [8]:
#Recursive character text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

re_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 200
)

In [11]:
doc_list = []
for page in cleaned_data:
  pg_split = re_splitter.split_text(page.page_content)

  for pg_sub_split in pg_split:
    metadata = {"source": "AI policy", "page_no": page.metadata["page"] + 1}

    doc_string = Document(page_content = pg_sub_split, metadata=metadata)
    doc_list.append(doc_string)

In [37]:
doc_list[91]

IndexError: list index out of range

In [16]:
print(len(doc_list))

198


## **Embeddings model**

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings
import os

In [20]:
#Intialize embedding model
embed_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')

/tmp/ipython-input-1714200968.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **Pinecone Vector Store**

In [21]:
%pip install -qU langchain-pinecone pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 9.9 MB/s eta 0:00:00


 **Create a Index in Pinecone**

In [23]:
import getpass
import os

from pinecone import Pinecone

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

Enter your Pinecone API key: ··········


In [24]:
from pinecone import ServerlessSpec

index_name = "rag-ai-policy"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

**Pinecone credentials**

In [ ]:
# PINECONE_API_KEY = "Your API Key"
index_name = "rag-ai-policy"

### **Data Upsertion in Pinecone**

In [27]:
from langchain_pinecone import PineconeVectorStore as langchain_pinecone
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [28]:
#Convert documents into vectors using LangPinecone
vector = langchain_pinecone.from_documents(
    doc_list,
    embed_model,
    index_name = index_name
)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## **Query**

In [ ]:
query = "What is AI policy for students"
pinecone_results = vector.similarity_search(query, k=1)

In [38]:
for result in pinecone_results:
  print(result.page_content)

and allied technologies through fiscal, legal, and technical support. b. The CoE -AI shall aim to facilitate the registration of at least 400 patents in high -tech for academia and the private sector annually. 4.2.3 Algorithms, Data Science & AI in Basic Education I. Where the policy document emphasizes the fundamental understanding and awareness of personal data protection and AI, it also aims to stimulate an incremental impact of AI on society right from the grassroots. Therefore, the policy has given equal importance to teaching algorithms, data science, and AI in basic STEM education. In this regard, it stipulates that. CoE-AI shall hire a local/international consultant with expertise in high -tech curriculum development to develop a National Curriculum in Algorithms, Data Sciences,
